[2025-08-15 Fiddler](https://thefiddler.substack.com/p/how-much-free-money-can-you-win)
====================

Fiddler
-------
One way to guarantee \\$35 in winnings is to bet \\$10 on each side, then
bet \\$25 on one side and \\$20 on the other side.  The possible outcomes
are
* \\$35 in winnings with \\$25 in vouchers
* \\$30 in winnings with \\$20 in vouchers

In the first case, the only possible bet is \\$25 on one side, so there's a
50% chance at \\$35 in winnings and a 50% chance at \\$60 in winnings.

In the second case, betting \\$10 on each side, then betting the last \\$10
voucher on one side gives a 50% chance at \\$40 in winnings and a 50% chance
at \\$50 in winnings.

Extra credit
------------
A lower bound is \\$90, by betting everything on one side with a 50% chance of winning
\\$55 and then taking the guaranteed \\$35.

If you hedge first, you can't make as big an all or nothing bet later.

If you hedge first, then do all or nothing bets on both branches of the second bet,
the best you can do is a 50% chance at \\$55 by betting \\$10 on either side for the
first bet.  Every other hedge results in a 75% chance of winning less than \\$55.

I doubt that more hedging could do better than \\$90, but haven't been able to prove it.
It's not obvious how to use brute force code, since there is the possibility of winning
every bet when betting on only one side.

In [1]:
def bets(vouchers):
    if vouchers == 55:
        return [(10,0),(20,0),(25,0),(30,0),
                (35,0),(45,0),(55,0),(10,10),
                (20,10),(25,10),(35,10),(45,10),
                (25,20),(35,20),(30,25)]
    elif vouchers == 45:
        return [(10,0),(20,0),(25,0),(35,0),(45,0),
                (10,10),(25,10),(35,10),(25,20)]
    elif vouchers == 35:
        return [(10,0),(25,0),(35,0),(10,25)]
    elif vouchers == 30:
        return [(10,0),(20,0),(30,0),(10,10),(20,10)]
    elif vouchers == 25:
        return [(25,0)]
    elif vouchers == 20:
        return [(10,0),(20,0),(10,10)]
    elif vouchers == 10:
        return [(10,0)]
    else:
        return []

In [2]:
def guaranteed(winnings,vouchers,history):
    if vouchers == 10 or vouchers == 25:
        return (winnings,history)
    new_winnings = winnings
    new_history = history
    for bet in bets(vouchers):
        (bet1,bet2) = bet
        if bet1 != 0 and bet2 != 0:
            (g1,h1) = guaranteed(winnings+bet1,vouchers-bet2,history+[bet])
            (g2,h2) = guaranteed(winnings+bet2,vouchers-bet1,history+[bet])
            g = min(g1, g2)
            if g > new_winnings:
                new_winnings = g
                new_history = h1
    return (new_winnings,new_history)

In [3]:
guaranteed(0,55,[])

(35, [(10, 10), (25, 20)])

My original reasoning that \\$30 was the minimum guaranteed winnings was
wrong.

In [4]:
def next_states(states):
    result = []
    for state in states:
        (p,history,ahistory,winnings,vouchers) = state
        betset = frozenset(bets(vouchers))
        if len(betset) == 0:
            result.append(state)
        else:
            for bet in betset:
                (bet1,bet2) = bet
                next_history = history | frozenset([(bet,winnings,vouchers)])
                next_ahistory = ahistory
                for b in betset:
                    if b != bet:
                        next_ahistory = next_ahistory | frozenset([(b,winnings,vouchers)])
                if bet1 == bet2:
                    result.append((p,next_history,next_ahistory,winnings+bet1,vouchers-bet2))
                else:
                    result.append((p/2,next_history,next_ahistory,winnings+bet1,vouchers-bet2))
                    result.append((p/2,next_history,next_ahistory,winnings+bet2,vouchers-bet1))
    return result

In [5]:
len(next_states(next_states(next_states(next_states([(1,frozenset(),frozenset(),0,55)])))))

34739

There are 34739 ways to make up to 4 bets, where making fewer bets
is only due to losing all the vouchers.

There are 15 possible first bets, so segregating the ways by first bet gives

In [6]:
results = []
for bet in bets(55):
    (bet1,bet2) = bet
    if bet1 == bet2:
        states = [(1,frozenset(),frozenset(),bet1,55-bet2)]
    else:
        states = [(1/2,frozenset(),frozenset(),bet1,55-bet2),
                  (1/2,frozenset(),frozenset(),bet2,55-bet1)]
    results.append(len(next_states(next_states(next_states(states)))))
results, sum(results)

([4898,
  4115,
  4196,
  4007,
  4040,
  4007,
  4004,
  895,
  1007,
  1088,
  932,
  899,
  305,
  149,
  197],
 34739)

4898 is a lot more reasonable to brute force than 34739.  Since outcomes of different
first bets can never be combined, while outcomes of different second or later bets could
be combined if they come from different outcomes of the first bet or another earlier
bet.

In [7]:
def best_combination(states, historys, ahistorys, first_index, p_threshold, min_winnings):
    for i in [first_index..len(states)-1]:
        (p,history,ahistory,winnings,vouchers) = states[i]
        if len(historys & ahistory) > 0 or len(ahistorys & history) > 0:
            continue
        alternative = best_combination(states, historys, ahistorys, i+1, p_threshold, min_winnings)
        if min_winnings == None or min_winnings > winnings:
            min_winnings = winnings
        if p < p_threshold:
            min_winnings = best_combination(states, history|historys, ahistory|ahistorys,
                                            i+1, p_threshold-p, min_winnings)
        if alternative != None and alternative > min_winnings:
            return alternative
        else:
            return min_winnings
    return None

In [8]:
best_combination(next_states(next_states(next_states([(1/2,frozenset(),frozenset(),30,30),
                                                      (1/2,frozenset(),frozenset(),25,25)]))),
                 frozenset(),frozenset(), 0, 1/2, None)

60

This calculation of combinations of 197 outcomes took about 5 seconds on my computer.

In [9]:
best_combination(next_states(next_states(next_states([(1/2,frozenset(),frozenset(),35,35),
                                                      (1/2,frozenset(),frozenset(),20,20)]))),
                 frozenset(),frozenset(), 0, 1/2, None)

55

And this calculation of combinations of just 149 outcomes took close to 30
seconds on my computer, so a brute force calculation of 4898 seems unreasonable.

### More thoughts
Consider a simplified scenario with 2 vouchers for $a$ and $b$ where
$a \le b$.  There are 4 possible first bets.

Consider the simplest case, betting $a+b$ on one side.  There is a 50%
chance of winning nothing, so $W$ is $a+b$ plus whatever the guaranteed
winnings can be taken with the two vouchers, which is $a$, so $W$ is $2a+b$.

The second simplest case, betting $a$ vs. $b$.  On one hand, there is a
50% chance of winning at least $a$, 25% chance of winning at least $2a$.
On the other hand, there is a 50% chance of winning at least $b$, 25% chance
of winning at least $2b$.  If $2a < b$, then $W = b$, otherwise $W = 2a$.
This is not as good at the first case.

The other two cases are betting one of the vouchers and withholding the
other for the first bet.  Consider betting $b$ first.  There is a 50%
chance of losing the first bet, in which case, there is a 1/4 chance of
winning at least $a$, which is irrelevant, since there is already a 50%
chance of winning at least $b$.  So, in the end, taking the guaranteed
additional $a$ after winning the first bet, $W = a+b$.

In the last case, betting $a$ first, there is a 50% chance of losing,
giving a 1/4 chance of winning at least $b$.  And, when winning, if $2a \ge b$,
then $W = 2a$, otherwise $W = b$ by combining the 1/4 chance of winning at
least $b$ with a 1/4 chance of winning at least $a+W$.

This supports the idea that the strategy should be to bet all the vouchers
on one side for the first bet, then, if the first bet wins, going for the
guaranteed winnings with subsequent bets.

### Another approach
Generalize $W$ to $W_{55}$, where $W_{55}$ is $W$ when starting with \\$55 in
vouchers.

There are also $W_{45}$, $W_{35}$, $W_{30}$, $W_{25} = 25$, $W_{20}$, $W_{10} = 10$, and
$W_0 = 0$.

Also, let $G_{55}$, $G_{45}$, etc be the guaranteed winnings when starting with
\\$55, \\$45, etc in vouchers.

$$\begin{aligned}
    G_{0} &= 0 \\
    G_{10} &= 0 \\
    G_{20} &= 10 \\
    G_{25} &= 0 \\
    G_{30} &= 20 \\
    G_{35} &= 10 \\
    G_{45} &= 20 \\
    G_{55} &= 35 \\
\end{aligned}$$

For $W_{20}$, $a = 10$, $b = 10$, so $W_{20} = 30$.

For $W_{30}$, the possible first bets are 10-0, 20-0, 30-0, 10-10, and 20-10.
* Betting 10-0:
  * 1/4 chance of $W_{20} = 30$ or 1/2 chance of $G_{20} = 10$
  * 1/4 chance of $10+W_{30}$ or 1/2 chance of $10+G_{30} = 30$
  * $W_{30} \ge 30$
* Betting 20-0:
  * 1/4 chance of $W_{10} = 10$ or 1/2 chance of $G_{10} = 0$
  * 1/4 chance of $20+W_{30}$ or 1/2 chance of $20+G_{30} = 40$
  * $W_{30} \ge 40$
* Betting 30-0:
  * 1/2 chance of 0
  * 1/2 chance of $30+G_{30} = 50$
  * $W_{30} \ge 50$
* Betting 10-10:
  * 1/2 chance at $10+W_{20} = 30$
  * $W_{30} \ge 30$
* Betting 20-10:
  * 1/4 chance at $10+W_{10} = 20$ or 1/2 chance at $10+G_{10} = 10$
  * 1/4 chance at $20+W_{20} = 40$ or 1/2 chance at $20+G_{20} = 30$
  * $W_{30} \ge 30$
* $W_{30} = 50$.

For $W_{35}$, $a = 10$, $b = 25$, so $W_{35} = 45$.

For $W_{45}$, the possible first bets are 10-0, 20-0, 25-0, 35-0, 45-0, 10-10,
20-10, 25-10, 35-10, 25-20.
* Betting 10-0:
  * 1/4 chance of $W_{35} = 45$ or 1/2 chance of $G_{35} = 10$
  * 1/4 chance of $10+W_{45}$ or 1/2 chance of $10+G_{45} = 30$
  * $W_{45} \ge 45$
* Betting 20-0:
  * 1/4 chance of $W_{25} = 25$ or 1/2 chance of $G_{25} = 0$
  * 1/4 chance of $20+W_{45}$ or 1/2 chance of $20+G_{45} = 40$
  * $W_{45} \ge 40$
* Betting 25-0:
  * 1/4 chance of $W_{20} = 30$ or 1/2 chance of $G_{20} = 10$
  * 1/4 chance of $25+W_{45}$ or 1/2 chance of $25+G_{45} = 45$
  * $W_{45} \ge 45$
* Betting 35-0:
  * 1/4 chance of $W_{10} = 10$ or 1/2 chance of $G_{10} = 0$
  * 1/4 chance of $35+W_{45}$ or 1/2 chance of $35+G_{45} = 55$
  * $W_{45} \ge 55$
* Betting 45-0:
  * 1/2 chance of 0
  * 1/2 chance of $45+G_{45} = 65$
  * $W_{45} \ge 65$
* Betting 10-10:
  * 1/2 chance of $10+W_{35} = 55$
  * $W_{45} \ge 55$
* Betting 20-10:
  * 1/4 chance of $10+W_{25} = 35$ or 1/2 chance of $10+G_{25} = 10$
  * 1/4 chance of $20+W_{35} = 65$ or 1/2 chance of $20+G_{35} = 30$
  * $W_{45} \ge 35$
* Betting 25-10:
  * 1/4 chance of $10+W_{20} = 40$ or 1/2 chance of $10+G_{20} = 20$
  * 1/4 chance of $25+W_{35} = 70$ or 1/2 chance of $25+G_{35} = 35$
  * $W_{45} \ge 40$
* Betting 35-10:
  * 1/4 chance of $10+W_{10} = 20$ or 1/2 chance of $10+G_{10} = 10$
  * 1/4 chance of $35+W_{35} = 80$ or 1/2 chance of $35+G_{35} = 45$
  * $W_{45} \ge 45$
* Betting 25-20:
  * 1/4 chance of $20+W_{20} = 50$ or 1/2 chance of $20+G_{20} = 30$
  * 1/4 chance of $25+W_{25} = 50$ or 1/2 chance of $25+G_{25} = 25$
  * $W_{45} \ge 50$
* $W_{45} = 65$.

For $W_{55}$, the possible first bets are 10-0, 20-0, 25-0, 30-0, 35-0, 45-0, 55-0, 10-10,
20-10, 25-10, 35-10, 45-10, 25-20, 35-20, 30-25.
* Betting 10-0:
  * 1/4 chance of $W_{45} = 65$ or 1/2 chance of $G_{45} = 20$
  * 1/4 chance of $10+W_{55}$ or 1/2 chance of $10+G_{55} = 45$
  * $W_{55} \ge 65$
* Betting 20-0:
  * 1/4 chance of $W_{35} = 35$ or 1/2 chance of $G_{35} = 10$
  * 1/4 chance of $20+W_{55}$ or 1/2 chance of $20+G_{55} = 55$
  * $W_{55} \ge 55$
* Betting 25-0:
  * 1/4 chance of $W_{30} = 50$ or 1/2 chance of $G_{30} = 20$
  * 1/4 chance of $25+W_{55}$ or 1/2 chance of $25+G_{55} = 60$
  * $W_{55} \ge 50$
* Betting 30-0:
  * 1/4 chance of $W_{25} = 25$ or 1/2 chance of $G_{25} = 0$
  * 1/4 chance of $30+W_{55}$ or 1/2 chance of $30+G_{55} = 65$
  * $W_{55} \ge 65$
* Betting 35-0:
  * 1/4 chance of $W_{20} = 30$ or 1/2 chance of $G_{20} = 10$
  * 1/4 chance of $35+W_{55}$ or 1/2 chance of $35+G_{55} = 70$
  * $W_{55} \ge 70$
* Betting 45-0:
  * 1/4 chance of $W_{10} = 10$ or 1/2 chance of $G_{10} = 0$
  * 1/4 chance of $45+W_{55}$ or 1/2 chance of $45+G_{55} = 80$
  * $W_{55} \ge 80$
* Betting 55-0:
  * 1/2 chance of 0
  * 1/2 chance of $55+G_{55} = 90$
  * $W_{55} \ge 90$
* Betting 10-10:
  * 1/2 chance of $10+W_{45} = 75$
  * $W_{55} \ge 75$
* Betting 20-10:
  * 1/4 chance of $10+W_{35} = 45$ or 1/2 chance of $10+G_{35} = 20$
  * 1/4 chance of $20+W_{45} = 85$ or 1/2 chance of $20+G_{45} = 40$
  * $W_{55} \ge 45$
* Betting 25-10:
  * 1/4 chance of $10+W_{30} = 60$ or 1/2 chance of $10+G_{30} = 30$
  * 1/4 chance of $25+W_{45} = 90$ or 1/2 chance of $25+G_{45} = 45$
  * $W_{55} \ge 60$
* Betting 35-10:
  * 1/4 chance of $10+W_{20} = 40$ or 1/2 chance of $10+G_{20} = 20$
  * 1/4 chance of $35+W_{45} = 100$ or 1/2 chance of $35+G_{45} = 55$
  * $W_{55} \ge 55$
* Betting 45-10:
  * 1/4 chance of $10+W_{10} = 20$ or 1/2 chance of $10+G_{10} = 10$
  * 1/4 chance of $45+W_{45} = 110$ or 1/2 chance of $45+G_{35} = 65$
  * $W_{55} \ge 65$
* Betting 25-20:
  * 1/4 chance of $20+W_{30} = 70$ or 1/2 chance of $20+G_{30} = 40$
  * 1/4 chance of $25+W_{35} = 60$ or 1/2 chance of $25+G_{35} = 35$
  * $W_{55} \ge 60$
* Betting 35-20:
  * 1/4 chance of $20+W_{20} = 50$ or 1/2 chance of $20+G_{20} = 30$
  * 1/4 chance of $35+W_{35} = 70$ or 1/2 chance of $25+G_{35} = 45$
  * $W_{55} \ge 50$
* Betting 30-25:
  * 1/4 chance of $25+W_{25} = 50$ or 1/2 chance of $25+G_{25} = 25$
  * 1/4 chance of $30+W_{30} = 80$ or 1/2 chance of $30+G_{30} = 50$
  * $W_{55} \ge 50$
* $W_{55} = 90$.